# Paqueterías y funciones necesarias

In [1]:
#Las siguientes paqueterías son requeridas para graficar, cargar y generar archivos y cálculos de álgebra lineal
using Plots
using DelimitedFiles
using LinearAlgebra

#Los siguientes archivos con funciones de Julia son requeridos
include("Operaciones_Basicas.jl")
include("Red_Dual.jl")
include("Separacion_Arreglos_X_Y.jl")
include("Generador_Vecindades_Pto_Arbitrario.jl")
include("voronoi.jl")
include("Algoritmo_Radio.jl")
include("Algoritmo_Voronoi.jl")
include("Algoritmo_Cluster_Principal.jl")
include("Algoritmo_Parche_Cuadrado.jl")
include("Dentro_Fuera_Poligono.jl")
include("Diccionario_Indices_Centroides_Voronoi.jl")
include("Algoritmo_Dinamica_FFD.jl")

multiple_Flies (generic function with 1 method)

#### Cargamos las funciones que determinan la trayectoria de entrada y salida de los polígonos de Voronoi

In [2]:
include("Esferas_Duras_Trayectoria_Recta.jl")

estado_Tras_Tiempo_Rectas (generic function with 1 method)

In [3]:
#We verify the size of the obstacle to consider
const Radio_Obstaculo = 0.01;

In [4]:
#Verificamos el tamaño del obstáculo a considerar
Radio_Obstaculo

0.01

#### Generamos las condiciones que determinan el tipo de arreglo cuasiperiódico que vamos a construir

In [5]:
NSides = 75; #Simetría rotacional del arreglo cuasiperiódico a generar
Error_Margin = 0; #Margen de error en los números enteros "n" asociados a los vectores estrella del MDG
Star_Vectors = [[BigFloat(1),0]]; #Arreglo que contendrá a los vectores estrella
for i in 1:(NSides-1)
    push!(Star_Vectors, [cos((2*i)*pi/NSides), sin((2*i)*pi/NSides)]); #Vértices del polígono de NSides Lados
end
Alphas_Array = fill(0.2, NSides); #Arreglo de las constantes Alfas del MDG
Average_Distance_Stripes = fill(NSides/2, NSides); #Arreglo con la separación promedio entre franjas

#Cargamos la información para generar los parches "circulares" alrededor de un punto arbitrario
Patch_Information = readdlm("Informacion_Parche_N_$(NSides).csv", BigFloat);
Reduction_Factor = 0.6; #Factor para generar un "radio seguro"

#### Definimos condiciones sobre el vuelo de una partícula

In [6]:
Long_Fly_Time = 5000000; #Tiempo máximo de vuelo que tendrá cada partícula
Number_Segments = 10; #Número de iteraciones de vuelos libres que se calcularán
Number_Free_Flies = 10; #Número de vuelos libres a calcularan por cada iteración

#### Obtenemos la primera colisión de una partícula y su velocidad tras la colisión

In [7]:
function first_Collition(Error_Margin, Average_Distance_Stripes, Star_Vectors, Alphas_Array, Patch_Information, Reduction_Factor, Long_Fly_Time)
    global Array_Free_Fly = []; #Array where the length of the particle's first free flight will be saved
    Initial_Position_Obstacle = [0.0, 0.0]; #Position of a particle after colliding with an obstacle
    Initial_Velocity_Obstacle = [0.0, 0.0]; #Velocity of a particle after colliding with an obstacle
    
    APoint_Initial = [0.0,0.0]; #Array that will contain the initial position of the particles
    Counter_Iterations = 0; #Counter for the number of attempts made to find a valid starting position.
    SL = 1e6; #Size of the half side of the square centered on the (0,0) where a starting position is sought.
    
    #We look for a valid starting position in 100 attempts, if it is not found, it starts at (0,0)
    while Counter_Iterations < 100
        Counter_Iterations += 1;
        Test_Point = punto_Arbitrario(SL); #We generate an arbitrary point inside the square centered on the (0,0)
        Exclusion_Radius = 1.0; #Radius of a circle centered on Test_Point to filter possible container polygons
        
        #We generate the local neighborhood within the circle with radius "Exclusion_Radius" around the Test_Point point
        Dual_Points_R = region_Local_Radio(Error_Margin, Exclusion_Radius, Average_Distance_Stripes, Star_Vectors, Alphas_Array, Test_Point);
        
        #We check if Test_Point falls into some obstacle centered on the vertices of the container polygon
        if posicion_Valida_Obstaculos_Circulares(Test_Point, Dual_Points_R)
            APoint_Initial = Test_Point; break #We redefine the starting point and close the While loop
        end
    end
    
    println("The initial position is $(APoint_Initial)")
    
    #Algorithm that calculates the trajectories of the different particles
    Initial_Position_Obstacle, Initial_Velocity_Obstacle = first_Fly(Patch_Information, Reduction_Factor, Average_Distance_Stripes, Star_Vectors, Alphas_Array, APoint_Initial, Long_Fly_Time, Array_Free_Fly, Initial_Position_Obstacle, Initial_Velocity_Obstacle, cambio_Celda_Esferas_Duras_Trayectoria_Recta, estado_Tras_Tiempo_Rectas);

    return Initial_Position_Obstacle, Initial_Velocity_Obstacle
end

first_Collition (generic function with 1 method)

In [8]:
Initial_Position_Obstacle, Initial_Velocity_Obstacle = first_Collition(Error_Margin, Average_Distance_Stripes, Star_Vectors, Alphas_Array, Patch_Information, Reduction_Factor, Long_Fly_Time);

The initial position is [23050.75813935753, -990878.5708634449]
La velocidad inicial es [0.8524064419477231, 0.5228797736822711]


#### Obtengamos las subsecuentes colisiones de la partícula

In [9]:
function multiple_Collisions(Number_Segments, Number_Free_Flies, Average_Distance_Stripes, Star_Vectors, Alphas_Array, Patch_Information, Reduction_Factor, Long_Fly_Time, Initial_Position_Obstacle, Initial_Velocity_Obstacle)
    Free_Fly_Global = []; #Arreglo donde irán todos los tiempos de vuelos libres

    for δ in 1:Number_Segments
        open("Estado_FFD_N_$(NSides)_Vuelos_Libres_R0_01.txt", "w") do file
            write(file, "Estamos calculando la iteración $(δ).")
        end
        #Iteramos sobre el número de colisiones con obstáculos que buscamos
        for γ in 1:Number_Free_Flies
            tiempo1 = time()

            Array_Free_Fly = []; #Arreglo donde se guardará la longitud del vuelo libre de la partícula
            #Definimos como posición inicial la última posición al colisionar con un obstáculo.
            APoint_Initial = copy(Initial_Position_Obstacle);

            Initial_Position_Obstacle, Initial_Velocity_Obstacle = multiple_Flies(Patch_Information, Reduction_Factor, Average_Distance_Stripes, Star_Vectors, Alphas_Array, APoint_Initial, Long_Fly_Time, Array_Free_Fly, Initial_Position_Obstacle, Initial_Velocity_Obstacle, cambio_Celda_Esferas_Duras_Trayectoria_Recta, estado_Tras_Tiempo_Rectas)

            #Guardamos la longitud del vuelo libre que acabamos de realizar
            Free_Fly_Global = vcat(Free_Fly_Global, Array_Free_Fly)
            tiempo2 = time()
            println(tiempo2 - tiempo1)
        end

        writedlm("FFD_N_$(NSides)_Vuelos_Libres_R0_01.csv", Free_Fly_Global)
    end
    
    return Free_Fly_Global
end

multiple_Collisions (generic function with 1 method)

In [10]:
Array_Free_Flies = multiple_Collisions(Number_Segments, Number_Free_Flies, Average_Distance_Stripes, Star_Vectors, Alphas_Array, Patch_Information, Reduction_Factor, Long_Fly_Time, Initial_Position_Obstacle, Initial_Velocity_Obstacle);

2.0839998722076416
34.05200004577637
2.0290000438690186
23.73799991607666
6.5940001010894775
4.4720001220703125
17.58299994468689
6.664999961853027
8.20799994468689
29.320000171661377
2.0880000591278076
6.636999845504761
8.414000034332275
2.111999988555908
6.5899999141693115
2.1069998741149902
10.86300015449524
4.322000026702881
6.269999980926514
14.764000177383423
15.077000141143799
4.230999946594238
2.5320000648498535
10.366000175476074
6.346999883651733
6.299999952316284
2.1529998779296875
4.442000150680542
2.1059999465942383
4.2230000495910645
2.2619998455047607
8.772000074386597
15.025000095367432
8.59500002861023
26.52900004386902
14.740000009536743
6.3379998207092285
12.655999898910522
2.133000135421753
18.092000007629395
3.4110000133514404
6.940999984741211
2.25600004196167
2.246000051498413
3.0959999561309814
13.75
8.282999992370605
4.199000120162964
12.565999984741211
15.505000114440918
4.302000045776367
6.4049999713897705
10.543999910354614
12.296000003814697
8.5069999694824